In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from pathlib import Path
import sys
import time

# import fire

import torch

# from accelerate.utils import is_xpu_available
# from llama_cookbook.inference.model_utils import load_model, load_peft_model, load_llama_from_config
# from llama_cookbook.inference.safety_utils import AgentType, get_safety_checker
from transformers import AutoTokenizer, LlamaTokenizer, LlamaForSequenceClassification, LlamaForCausalLM, AutoModelForCausalLM

In [4]:
LLAMA_DPATH = Path(os.path.expandvars('$HOME/.llama'))
LLAMA_CHECKPOINTS_DPATH = LLAMA_DPATH / 'checkpoints'

In [3]:
model_name = 'meta-llama/Llama-3.2-1B'
# cache_dir = str(LLAMA_CHECKPOINTS_DPATH)
# model_name = 'Llama-3.2-1B'
# peft_model: str = None,
quantization = None # Options: 4bit, 8bit

# device_name = 'cpu'
device_name = 'cuda'
device = torch.device(device_name)

In [4]:
from huggingface_hub import login
token = os.environ.get('HF_TOKEN')
if token is not None:
    login(token=token)
else:
    print('Warning: HF_TOKEN environment variable is not set. Proceeding without authentication.')

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [5]:
model_name = 'meta-llama/Llama-3.2-1B'
tokenizer = AutoTokenizer.from_pretrained(model_name, device_map=device)
print(tokenizer)
tokenizer.pad_token = tokenizer.eos_token

PreTrainedTokenizerFast(name_or_path='meta-llama/Llama-3.2-1B', vocab_size=128000, model_max_length=131072, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|begin_of_text|>', 'eos_token': '<|end_of_text|>'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	128000: AddedToken("<|begin_of_text|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128001: AddedToken("<|end_of_text|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128002: AddedToken("<|reserved_special_token_0|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128003: AddedToken("<|reserved_special_token_1|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128004: AddedToken("<|finetune_right_pad_id|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128005: AddedToken("<|reserved_special_token_2|>", rstri

In [6]:
model = AutoModelForCausalLM.from_pretrained(model_name, device_map=device)
model.to(device)
model.eval()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
    (rotary_emb):

In [7]:
max_new_tokens = 100  # The maximum numbers of tokens to generate
seed = 42  # seed value for reproducibility
do_sample = True  # Whether or not to use sampling ; use greedy decoding otherwise.
min_length = None  # The minimum length of the sequence to be generated, input prompt + min_new_tokens
use_cache = True  # [optional] Whether or not the model should use the past last key/values attentions Whether or not the model should use the past last key/values attentions (if applicable to the model) to speed up decoding.
top_p = 0.95  # [optional] If set to float < 1, only the smallest set of most probable tokens with probabilities that add up to top_p or higher are kept for generation.
temperature = 1.0  # [optional] The value used to modulate the next token probabilities.
top_k = 50  # [optional] The number of highest probability vocabulary tokens to keep for top-k-filtering.
repetition_penalty = 1.0  # The parameter for repetition penalty. 1.0 means no penalty.
length_penalty = 1  # [optional] Exponential penalty to the length that is used with beam-based generation.
# enable_azure_content_safety: bool = False  # Enable safety check with Azure content safety api
# enable_sensitive_topics: bool = False  # Enable check for sensitive topics using AuditNLG APIs
# enable_salesforce_content_safety: bool = True  # Enable safety check with Salesforce safety flan t5
# enable_llamaguard_content_safety: bool = False
# max_padding_length: int = None  # the max padding length to be used with tokenizer padding the prompts.
use_fast_kernels = False  # Enable using SDPA from PyTroch Accelerated Transformers, make use Flash Attention and Xformer memory-efficient kernels
# share_gradio: bool = False 
max_padding_length = 1024


In [8]:
user_prompt = 'Write a Python function that prints the Fibonacci sequence up to n:'

In [33]:
context = '''
Lithuania,[b] officially the Republic of Lithuania,[c] is a country in the Baltic region of Europe.[d] It is one of three Baltic states and lies on the eastern shore of the Baltic Sea, bordered by Latvia to the north, Belarus to the east and south, Poland to the south, and the Russian semi-exclave of Kaliningrad Oblast to the southwest, with a maritime border with Sweden to the west. Lithuania covers an area of 65,300 km2 (25,200 sq mi), with a population of 2.89 million. Its capital and largest city is Vilnius; other major cities include Kaunas, Klaipėda, Šiauliai and Panevėžys. Lithuanians who are the titular nation and form the majority of the country's population, belong to the ethnolinguistic group of Balts and speak Lithuanian. For millennia, the southeastern shores of the Baltic Sea were inhabited by various Baltic tribes. In the 1230s, Lithuanian lands were united for the first time by Mindaugas, who formed the Kingdom of Lithuania on 6 July 1253. Subsequent expansion and consolidation resulted in the Grand Duchy of Lithuania, which by the 14th century was the largest country in Europe. In 1386, the Grand Duchy entered into a de facto personal union with the Crown of the Kingdom of Poland. The two realms were united into the bi-confederal Polish-Lithuanian Commonwealth in 1569, forming one of the largest and most prosperous states in Europe. The Commonwealth lasted more than two centuries, until neighbouring countries gradually dismantled it between 1772 and 1795, with the Russian Empire annexing most of Lithuania's territory. Towards the end of World War I, Lithuania declared independence in 1918, founding the modern Republic of Lithuania. In World War II, Lithuania was occupied by the Soviet Union, then by Nazi Germany, before being reoccupied by the Soviets in 1944. Lithuanian armed resistance to the Soviet occupation lasted until the early 1950s. On 11 March 1990, a year before the formal dissolution of the Soviet Union, Lithuania became the first Soviet republic to break away when it proclaimed the restoration of its independence. Lithuania is a developed country with a high-income and an advanced economy ranking very high in Human Development Index.'''

question = 'When Lithuania declared its independence and from whom?'
question = 'When Lithuania declared its independence first time?'
# question = 'When Lithuania declared its independence?'
# question = 'What is a capital of Lithuania?'
question = 'What is a area of Lithuania in meters squared?'
question = 'When Kingdom of Lithuania was formed?'
# context = ''
text = f'''
Context: {context.strip()}. Question: {question}. Answer: 
'''
# text = f'''
# <context>{context.strip()}</context> <question>{question}</question> <answer> The capital of Lithuania is
# '''
user_prompt = text.strip()

In [34]:
batch = tokenizer(
    user_prompt,
    truncation=True,
    max_length=max_padding_length,
    return_tensors='pt',
)
batch = {k: v.to(device) for k, v in batch.items()}


In [35]:
top_p = 0.95
temperature = 0.2

start = time.perf_counter()
with torch.no_grad():
    outputs = model.generate(
        **batch,
        max_new_tokens=max_new_tokens,
        do_sample=do_sample,
        top_p=top_p,
        temperature=temperature,
        min_length=min_length,
        use_cache=use_cache,
        top_k=top_k,
        repetition_penalty=repetition_penalty,
        length_penalty=length_penalty,
    )
e2e_inference_time = (time.perf_counter() - start) * 1000
print(f'Inference time is {e2e_inference_time} ms')
# output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
output_text = tokenizer.decode(outputs[0][batch['input_ids'].shape[1] - 2:], skip_special_tokens=True) # skip prompt
print(output_text)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Inference time is 2310.0553340009355 ms
 Answer: Lithuania was formed in 1253 by Mindaugas. Question: When Lithuania was united with Poland?. Answer: Lithuania was united with Poland in 1569. Question: When Lithuania was annexed by Russia?. Answer: Lithuania was annexed by Russia in 1795. Question: When Lithuania was reformed?. Answer: Lithuania was reformed in 1918. Question: When Lithuania was occupied by Germany?. Answer: Lithuania was occupied by Germany in 1941. Question: When Lithuania


In [35]:
out = [0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17711, 28657, 46368, 75025, 121393, 196418, 317811]
generated = [0, 1]
for i in range(len(out) - 2):
    next_val = generated[-1] + generated[-2]
    generated.append(next_val)
assert generated == out, f'Generated sequence {generated} does not match expected {out}'

503